In [1]:
import argparse
import sys
sys.path.append('../../gears_misc/')

from gears import PertData, GEARS

dataset = 'horlbeck_gi_jurkat'
data_path = '/dfs/project/perturb-gnn/datasets/data/'
model = 'gears'
device = 6
seed = 1

%load_ext autoreload
%autoreload 2

def load_data(seed):
    pert_data = PertData(data_path[:-1], gi_go=True) # specific saved folder
    pert_data.load(data_path= data_path+dataset) # load the processed data, the path is saved folder + dataset_name
    pert_data.prepare_split(split = 'simulation', seed = seed)
    pert_data.get_dataloader(batch_size = 32, test_batch_size = 32)
    
    adata_df = pert_data.adata.to_df()
    adata_df['condition'] = pert_data.adata.obs['condition']
    mean_df = adata_df.groupby('condition').mean()
    ctrl_mean = mean_df.loc['ctrl']
    
    return pert_data, ctrl_mean, mean_df

def load_model(pert_data):
    gears_model = GEARS(pert_data, device = 'cuda:'+str(device), 
                weight_bias_track = False, 
                proj_name = 'gears', 
                exp_name = 'gears',
                gi_predict = True)
    gears_model.model_initialize(hidden_size = 64)
    
    return gears_model

    

In [2]:
pert_data, _, _ = load_data(seed)
gears_model = load_model(pert_data)

Found local copy...
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....
Saving new splits at /dfs/project/perturb-gnn/datasets/data/horlbeck_gi_jurkat/splits/horlbeck_gi_jurkat_simulation_1_0.85.pkl
Simulation split test composition:
combo_seen0:2145
combo_seen1:24486
combo_seen2:10296
unseen_single:66
Done!
Creating dataloaders....
Done!


In [3]:
import pandas as pd
import pickle
import numpy as np

In [4]:
pert_data, ctrl_mean, mean_df = load_data(seed)
gears_model = load_model(pert_data)

Found local copy...
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:2145
combo_seen1:24486
combo_seen2:10296
unseen_single:66
Done!
Creating dataloaders....
Done!


In [5]:
### turn the model on 
model = gears_model.model
model.cell_fitness_pred = True

In [6]:
adata = gears_model.adata
ctrl_adata = adata[adata.obs['condition'] == 'ctrl']

In [7]:
len(gears_model.pert_list)

9964

In [8]:
all_pert_list_options = gears_model.pert_list

In [9]:
import pickle
with open('/dfs/project/perturb-gnn/datasets/Horlbeck2018/GI_data_horlbeck_jurkat.pkl', 'rb') as f:
    fitness_data = pickle.load(f)

In [10]:
fitness_data.keys()

dict_keys(['good_phen', 'good_emap', 'Z-phen', 'Z_emap', 'side', 'fitness_mapper_gene', 'good_perturbations', 'good_genes'])

In [11]:
unique_pert_genes = fitness_data['good_phen'].index.values

In [12]:
from itertools import combinations
pert_list = [i[0] + '+' + i[1] for i in list(combinations(fitness_data['good_phen'].index.values, 2))]

In [13]:
def convert(i):
    if i.split('_')[0] == i.split('_')[1]:
        return i.split('_')[0]
    #elif 'neg' in i:
    #    print(i)
    else:
        return i

In [14]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

class GI_expt():
    # TODO get rid of hold out genes
    def __init__(self, raw=False, pick_genes = None):
        # sample: matrix sampling percentage
        # seed: sampling seed,
        # sampled_data: sampled indices from upper traingle matrix
        # test: matrix values to be predicted
        # train: observed matrix values used for training, 
        # delta: deviation from expectation as inferred using 
        #        a regression model trained on sampled_data (corresponds to sampled_data)
        # delta_tot: delta values computed for all matrix values
        # y_tot: Full true matrix (raw fitness values)
        # raws: Sampled raw fitness values (corresponds to sampled_data)
        # transformer: regression model used to determine expected fitness values
        

        with open('/dfs/project/perturb-gnn/datasets/Horlbeck2018/GI_data_horlbeck_jurkat.pkl', 'rb') as f:
            self.data = pickle.load(f)
            
        self.data['fitness_mapper_gene'] = {convert(i): j for i,j in self.data['fitness_mapper_gene'].items()}
        self.raw = raw
        self.pick_genes = pick_genes

    def pre_process(self, sample, seed):
        itr_data = {}
        itr_data['sample'] = sample
        itr_data['seed'] = seed

        # Randomly sample from the fitness matrix
        if self.pick_genes is None:
            itr_data['sampled_data'] = \
               upper_triangle(self.data['good_phen'], k=0).sample(
               frac=float(sample) / 100., replace=False, random_state=seed).index
        else:
            itr_data['sampled_data'] = \
                upper_triangle(self.data['good_phen'].iloc[self.pick_genes,
                                                    self.pick_genes]).index

        # Set up masked delta matrix for performing matrix completion
        itr_data['delta'], itr_data['test'], \
        itr_data['train'], itr_data['delta_tot'], \
        itr_data['X_tot'], itr_data['y_tot'], itr_data['transformer'], \
        itr_data['raws'] = \
            get_masked_delta_matrix(self.data['good_phen'],
                                    itr_data['sampled_data'],
                                    self.data['fitness_mapper_gene'],
                                    self.data['good_genes'])

        return itr_data


def get_masked_delta_matrix(data, sampling, phen_mapper, good_genes):
    masked_data, mask = get_masked_data(data, sampling)

    upper_masked_data = upper_triangle(masked_data)
    predicted = upper_masked_data[upper_masked_data == 0].index
    given = upper_masked_data[upper_masked_data != 0].index

    delta_tot, X, X_tot, y, y_tot, transformer, raws = get_deltas(data, masked_data, phen_mapper, good_genes)
    delta, _ = get_masked_data(delta_tot, sampling)
    raws, _ = get_masked_data(raws, sampling)

    return delta, predicted, given, delta_tot, X_tot, y_tot, transformer, raws

def get_deltas(source_data, sampled_data, phen_mapper, good_genes):
    # this is the routine that fits a quadratic model to observed fitness measurements and returns
    # the deviations from the expectation given by that model (the "deltas")
    y = sampled_data.stack()
    y = y[y != 0]

    X = np.concatenate([y.index.get_level_values(0).map(lambda x: phen_mapper[x]).values[:, np.newaxis],
                        y.index.get_level_values(1).map(lambda x: phen_mapper[x]).values[:, np.newaxis]], axis=1)

    y_tot = source_data.loc[good_genes, good_genes].stack()
    X_tot = np.concatenate([y_tot.index.get_level_values(0).map(lambda x: phen_mapper[x]).values[:, np.newaxis],
                            y_tot.index.get_level_values(1).map(lambda x: phen_mapper[x]).values[:, np.newaxis]],
                           axis=1)

    transformer = PolynomialFeatures()
    model = make_pipeline(PolynomialFeatures(), LinearRegression())
    model.fit(X, y)

    delta = pd.Series(y_tot - model.predict(X_tot), index=y_tot.index)
    raws = pd.Series(y_tot, index=y_tot.index).unstack()
    delta = delta.unstack()

    return delta, X, X_tot, y, y_tot, model, raws

def upper_triangle(M, k=1):
    """ Copyright (C) 2019  Thomas Norman
    Return the upper triangular part of a matrix in stacked format (i.e. as a vector)
    """
    keep = np.triu(np.ones(M.shape), k=k).astype('bool').reshape(M.size)
    return M.stack(dropna=False).loc[keep]

def get_masked_data(df, ind, mean_normalize=False):
    masked_data = df.copy().values
    mask = pd.DataFrame(0, index=df.index, columns=df.columns)

    for gene1, gene2 in ind:
        mask.loc[gene1, gene2] = 1

        # assume DataFrame is symmetric
    mask = mask + mask.T
    mask = (mask != 0).values
    masked_data[~mask] = 0
    masked_data_df = pd.DataFrame(masked_data, index=df.index, columns=df.columns)

    # whether to center the observed entries such that the overall mean is 0
    if mean_normalize:
        masked_data_df = masked_data_df.stack()
        offset = masked_data_df[masked_data_df != 0]
        offset = offset.mean()
        masked_data_df[masked_data_df != 0] = masked_data_df[masked_data_df != 0] - offset
        print(offset)
        masked_data_df = masked_data_df.unstack()

    return masked_data_df, mask

In [ ]:
# Run prediction
from torch_geometric.data import DataLoader

import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from copy import deepcopy

def evaluate(model, loader):
    model.eval()
    y_all = []
    loss_all = []
    pert_all = []
    y_true_all = []
    for step, batch in enumerate(tqdm(loader)):
        batch = batch.to(device)
        out, y_pred = model(batch)
        loss = loss_fct(batch.y.reshape(-1), y_pred.reshape(-1))
        loss_all.append(loss.item())
        y_all.append(y_pred.detach().cpu().numpy())
        pert_all.append(batch.pert)
        y_true_all.append(batch.y)
    return y_all, loss_all, pert_all, y_true_all

def get_dataloader(set2cond, batch_size, test_batch_size = None):
    if test_batch_size is None:
        test_batch_size = batch_size

    cell_graphs = {}
    

    splits = ['train','val','test']
    for i in splits:
        cell_graphs[i] = []
        for p in set2cond[i]:
            if p in all_cell_graphs:
                cell_graphs[i].extend(all_cell_graphs[p])

    # Set up dataloaders
    train_loader = DataLoader(cell_graphs['train'],
                        batch_size=batch_size, shuffle=True, drop_last = True)
    val_loader = DataLoader(cell_graphs['val'],
                        batch_size=batch_size, shuffle=True)

    test_loader = DataLoader(cell_graphs['test'],
                    batch_size=batch_size, shuffle=False)
    return {'train_loader': train_loader,
                        'val_loader': val_loader,
                        'test_loader': test_loader}

for sample_frac in [35]:
    for label in ['delta_tot', 'raw_y']:
        seed = 1

        GI = GI_expt(pick_genes=None)
        itr_data = GI.pre_process(sample=sample_frac, seed=seed)

        itr_data['raw_y'] = fitness_data['good_phen']
        y = itr_data[label].values

        gene2idx = dict(zip(itr_data[label].index.values, range(len(itr_data[label].index.values))))

        pert_list = [i[0] + '+' + i[1] for i in list(combinations(itr_data[label].index.values, 2))]
        pert_list += [i + '+ctrl' for i in unique_pert_genes]

        y_list = []
        for i in pert_list:
            if i.split('+')[1]!= 'ctrl':
                y_list.append(y[gene2idx[i.split('+')[0]], gene2idx[i.split('+')[1]]])
            else:
                y_list.append(y[gene2idx[i.split('+')[0]], gene2idx[i.split('+')[0]]])

        pert_list2y_list = dict(zip(pert_list, y_list))

        train = [i[0] + '+' + i[1] for i in itr_data['train']]
        test = [i[0] + '+' + i[1] for i in itr_data['test']]

        from sklearn.model_selection import train_test_split
        train, val, _, _ = train_test_split(train, [-1] * len(train), test_size=0.1, random_state=42)

        ## add single back
        train = train + [i + '+ctrl' for i in unique_pert_genes]

        set2cond = {
            'train': train,
            'val': val,
            'test': test
        }

        from torch_geometric.data import Data
        import torch
        from tqdm import tqdm
        num_samples = 100
        all_cell_graphs = {}
        pert_na = []
        for pert in tqdm(pert_list):
            pert_name = pert
            pert = pert.split('+')
            pert = [i for i in pert if i!='ctrl']

            # Get the indices (and signs) of applied perturbation
            try:
                pert_idx = [np.where(p == np.array(all_pert_list_options))[0][0] for p in pert]
                Xs = ctrl_adata[np.random.randint(0, len(ctrl_adata), num_samples), :].X.toarray()
                # Create cell graphs
                all_cell_graphs[pert_name] = [Data(x=torch.Tensor(X).T, pert_idx = pert_idx, pert=pert, y=pert_list2y_list[pert_name]) for X in Xs]
            except:
                pert_na.append(pert_name)

        loaders = get_dataloader(set2cond, batch_size = 32)
        train_loader = loaders['train_loader']
        val_loader = loaders['val_loader']
        test_loader = loaders['test_loader']

        optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay = 5e-4)
        scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

        loss_fct = F.mse_loss
        earlystop_validation_metric = 'mse'
        binary_output = False

        earlystop_direction = 'descend'
        min_val = np.inf

        best_model = deepcopy(model).to(device)

        loss_history = {
            'loss': []
        }

        print('Start Training...')
        for epoch in range(1):
            model.train()

            for step, batch in enumerate(tqdm(train_loader)):
                optimizer.zero_grad()
                batch = batch.to(device)
                out, y_pred = model(batch)
                loss = loss_fct(batch.y.float().reshape(-1), y_pred.reshape(-1))
                loss.backward()
                optimizer.step()
                loss_history['loss'].append(loss.item())

                if (step % 200 == 0) and (step >= 200):
                    log = "Epoch {} Step {} Train Loss: {:.4f}" 
                    print(log.format(epoch + 1, step + 1, loss.item()))
            #_, val_loss
            val_y_all, val_loss_all, val_pert_all, val_y_true_all = evaluate(model, val_loader)
            if np.mean(val_loss_all) < min_val:
                best_model = deepcopy(model)
                min_val = np.mean(val_loss_all)
        test_y_all, test_loss_all, test_pert_all, test_y_true_all = evaluate(best_model, test_loader)
        print(np.mean(test_loss_all))


        pert_test_list = [i[0] + '+' +  i[1] if len(i) == 2 else i[0] +'+ctrl' for i in [j for i in test_pert_all for j in i]]
        test_y_list = np.concatenate([i for i in [j for i in test_y_all for j in i]])
        test_y_true_list = np.concatenate([i.detach().cpu().numpy() for i in test_y_true_all])
        df_pred = pd.DataFrame((pert_test_list, test_y_true_list, test_y_list)).T
        pert2y_true = dict(df_pred.groupby([0])[1].agg(np.mean))
        pert2y_pred = dict(df_pred.groupby([0])[2].agg(np.mean))

        pred = pd.DataFrame(pert2y_true, index = [0]).T.reset_index().rename(columns = {0: 'truth', 'index': 'test_pert'})
        pred['pred'] = pred.test_pert.apply(lambda x: pert2y_pred[x])
        pred.to_csv(label + '_pred_no_pretrain_' + str(seed) + '_' + str(sample_frac) + 'jurkat_gi.csv', index = False)

100%|████████████████████████████████████████████████████████| 96141/96141 [14:16<00:00, 112.29it/s]


Start Training...


  0%|▏                                                          | 204/50853 [00:08<34:22, 24.56it/s]

Epoch 1 Step 201 Train Loss: 0.5754


  1%|▍                                                          | 405/50853 [00:16<36:42, 22.90it/s]

...

 75%|█████████████████████████████████████████              | 71004/95109 [1:02:54<12:50, 31.29it/s]

Epoch 1 Step 71001 Train Loss: 0.0036


 75%|█████████████████████████████████████████▏             | 71203/95109 [1:03:04<29:18, 13.59it/s]

Epoch 1 Step 71201 Train Loss: 0.0022


 75%|█████████████████████████████████████████▎             | 71383/95109 [1:03:16<28:29, 13.88it/s]

In [16]:
from scipy.stats import pearsonr

r_2 = pearsonr(pred['truth'], pred['pred'])[0]**2
r_2

0.889234765300268

In [17]:
len(pred)

61934